In [1]:
from ipywidgets import FileUpload
from IPython.display import display
from ipywidgets import FileUpload
import gpxpy
import folium
from folium.plugins import TimestampedGeoJson

First select and upload a couple of GPX tracks

In [2]:
upload = FileUpload(accept='.gpx', multiple=True)
display(upload)

FileUpload(value={}, accept='.gpx', description='Upload', multiple=True)

In [6]:
def get_data(fname):
    points = []
    with open(fname, 'r') as f:
        gpx = gpxpy.parse(f)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append((point.latitude, point.longitude,
                                   point.speed, point.time))
    return points

Parse the uploaded files

In [7]:
tracks = [get_data(f) for f in upload.value]

Prepare a map

In [13]:
t = tracks[0]
latitude = sum(p[0] for p in t)/len(t)
longitude = sum(p[1] for p in t)/len(t)
my_map = folium.Map(location=[latitude, longitude], zoom_start=14)

In [14]:
colors = ['red', 'blue', 'green', 'brown', 'darkcyan']

data = {
    "type": "FeatureCollection",
    "features": []
}


def add_track(color, points, dt):
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': [[p[1], p[0]] for p in points]
        },
        'properties': {
            'times': [(p[3].timestamp() + dt) * 1000 for p in points],
            'style': {
                'color': color,
                'opacity': 0.5,
            },
        }
    }
    data['features'].append(feature)


for i, points in enumerate(tracks):
    dt = (tracks[0][0][3] - points[0][3]).total_seconds()
    add_track(colors[i], points, dt)

TimestampedGeoJson(data, period='PT5S', transition_time=50).add_to(my_map)
my_map